# A Brief Intro!

Langchain is a framework that simplifies the process of building powerful AI tools that can perform tasks like answering questions, processing documents, automating workflows, and much more. It allows you to integrate LLMs into your existing applications or build entirely new solutions with minimal effort, giving you the flexibility to develop customized AI solutions for a variety of use cases.

The "chain" in Langchain refers to sequence of steps or operations where different components, like language models, tools, and external resources, are connected to work together. Each step in the chain performs a specific task, such as calling an API, retrieving information, or processing data, and the output of one step can serve as the input to the next, hence the name "chain".

## Building With Langchain

The simplest and most common chain contains three things:

* LLM/Chat Model: The language model is the core reasoning engine here. In order to work with LangChain, you need to understand the different types of language models and how to work with them.
* Prompt Template: This provides instructions to the language model. This controls what the language model outputs, so understanding how to construct prompts and different prompting strategies is crucial.
* Output Parser: These translate the raw response from the language model to a more workable format, making it easy to use the output downstream.

## LLM/Chat Model

There are two types of language models within Langchain:

* LLM: underlying model takes a string as input and returns a string
* ChatModel: underlying model takes a list of messages as input and returns a message

In [1]:
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
import os, getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

A message unlike strings is an object of type 'BaseMessage' and is composed of two attributes

* content: The content of the message. Usually a string.
* role: The entity from which the BaseMessage is coming.

Langchain provides several types of messages to distinguish between the roles:

* HumanMessage: A BaseMessage coming from a human/user.
* AIMessage: A BaseMessage coming from an AI/assistant.
* SystemMessage: A BaseMessage coming from the system.
* FunctionMessage / ToolMessage: A BaseMessage containing the output of a function or tool call.

The simplest way to call an LLM or ChatModel is using .invoke()

* LLM.invoke: Takes in a string, returns a string.
* ChatModel.invoke: Takes in a list of BaseMessage, returns a BaseMessage.

Lets see how we can use the 'invoke' method to initiate a response from both types of AI models llm and chat model.


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

text_input = "how many usable host IP addresses are under subnet 192.168.10.0/24?"
system_message = """
As an expert in the field of computer networking answer all the user related questions. \
try to keep the answers short and concise.
"""
message = [HumanMessage(content=text_input), SystemMessage(content=system_message)]

llm = OpenAI()
chat_model = ChatOpenAI()

#llm.invoke(text_input)
chat_model.invoke(message)

AIMessage(content='In a /24 subnet, there are 256 total IP addresses. However, the first and last IP addresses are reserved for network address and broadcast address, leaving 254 usable host IP addresses. So, under subnet 192.168.10.0/24, there are 254 usable host IP addresses.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 57, 'total_tokens': 120, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0140cb80-8b67-4992-918d-86e92bae7c36-0', usage_metadata={'input_tokens': 57, 'output_tokens': 63, 'total_tokens': 120})

## Prompt Templates

Prompt templates are used to format user inputs to the LLMs. Instead of simply providing a text/string input like we did above, templates allow us to insert placeholders in to the text.

The value for these placeholders can then be provided at runtime when the llm is called. Lets see that with an example.

In [8]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("suggest a {name} for use on a computer networking router based on its location at {location}")

prompt.format(name="username", location="san francisco")
#prompt.invoke({"name": "username", "location": "san francisco"})

llm.invoke(prompt.format(name="username", location="san francisco"))

'\n\n"SFO_Router" '

Prompt templates are also available for chat models. For chat models we can put placeholders for inputs from each type of role.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

system_template = system_message
human_template = "suggest a {name} for use on a computer networking router located in {location}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_template)
])

chat_prompt.format_messages(name="username", location="san francisco")

chat_model.invoke(chat_prompt.format_messages(name="username", location="san francisco"))



AIMessage(content='"SFNetRouter"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 52, 'total_tokens': 57, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ae9bdef-a677-4e05-a24b-7e45a4791460-0', usage_metadata={'input_tokens': 52, 'output_tokens': 5, 'total_tokens': 57})

Langchain provides a lot of flexibility when it comes to formatting the prompts. Here's another way of constructing a similar chat promt using differnt types of message objects

In [13]:
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("As an expert in the field of computer networking answer all the user related questions. \
                     Try to keep the answers short and concise."
            )
        ),
        HumanMessagePromptTemplate.from_template("suggest a {name} for use on a computer networking router located in {location}"),
    ]
)

chat_model.invoke(chat_template.format_messages(name="username", location="san francisco"))

AIMessage(content='"SFNetMaster"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 52, 'total_tokens': 57, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a0a69107-0eee-496f-ba83-2da2fb16656c-0', usage_metadata={'input_tokens': 52, 'output_tokens': 5, 'total_tokens': 57})

## Output Parsers

Output from the LLMs are plain text but sometimes we desire to get a more structured output from our applications rather than just plain text.

This is where the output parsers are very helpful. In Langchain , Outputparsers are python interfaces that must implement the following two methods

* parse
* get_format_instructions

Lets write a very basic Output Parser and then go into a more common way of defining a Langchain Output parser

In [ ]:
from langchain_core.output_parsers import BaseOutputParser
from typing import List

class OutputListFormatter(BaseOutputParser):
    """ A class that formats the output from the LLM into a python list"""

    def parse(self, text_input: str) -> List[str]:
        return text_input.split(" ")

llm_response = llm.invoke("Describe weather forcast for today in San Francisco in two short sentences")
OutputListFormatter().parse(llm_response)


In the above example , we defined our own output parser. Langchain however has several pre-defined output parser classes that we can consume , instead of defining our own.

One of the popular output parser is "PydanticOutputParser" class. Let's look at an example of using this parser class.

In [16]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field

class WeatherForcast(BaseModel):
    query:str = Field(description="question asked to LLM Model")
    response:str = Field(description="response to the query from LLM model")

output_parser = PydanticOutputParser(pydantic_object=WeatherForcast)
prompt = PromptTemplate(
    template="{format_instructions}\nDescribe weather forcast for today at {location}\
          in no more than two short sentences",
    input_variables=["location"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )

llm(prompt.format(location="San Francisco"))


'\n\n{\n  "query": "Describe weather forcast for today at San Francisco",\n  "response": "Today in San Francisco, the weather will be mostly cloudy with a high of 64°F and a low of 55°F. There is a 20% chance of rain and winds of up to 10mph. Overall, it will be a cool and cloudy day."\n}'

So far we have been passing the prompt.format() method to create an instance of LLM, for. e.g "llm(prompt.format(location="San Francisco"))" used in the above cell.

Langchain also provides what is known as "Langchain Expression Language or LCEL" which uses the " | " operator to combine different components together, such as prompts and instance of LLM. LCEL is a way to create arbitrary custom chains.

Lets see that with an example. We are going to use the same "prompt" that we created in the previous cell.

In [ ]:
prompt_and_model_combined = prompt | llm
response = prompt_and_model_combined.invoke({"location": "San Francisco"})
print(response)

Just like we combined or chained prompt and llm, we can also combine output parser to form one single chain

In [ ]:
weather_chain = prompt | llm | output_parser
weather_chain.invoke({"location": "San Francisco"})